In [ ]:
import numpy
import cv2
import glob
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [30, 15]


sift = cv2.SIFT_create()
bf = cv2.BFMatcher()


template_paths = []
scene_paths = []
template_paths += glob.glob('./homework2/objets/*.jpg')
scene_paths += glob.glob('./homework2/images_test/*.jpg')

testing_image_number = 0

for scene_path in scene_paths:
    
    scene_bgr_image = cv2.imread(scene_path)
    scene_rgb_image = cv2.cvtColor(scene_bgr_image, cv2.COLOR_BGR2RGB)
    scene_gray_image = cv2.cvtColor(scene_bgr_image, cv2.COLOR_BGR2GRAY)
    scene_gray_image = cv2.GaussianBlur(scene_gray_image, (5, 5), 1)
    scene_keypoints, scene_descriptors = sift.detectAndCompute(scene_gray_image, None)

    
    max_template_matches = []
    max_template_rgb_image = None
    max_template_keypoints = None


    for template_path in template_paths:
        
        template_bgr_image = cv2.imread(template_path)
        template_rgb_image = cv2.cvtColor(template_bgr_image, cv2.COLOR_BGR2RGB)
        template_gray_image = cv2.cvtColor(template_bgr_image, cv2.COLOR_BGR2GRAY)
        template_gray_image = cv2.GaussianBlur(template_gray_image, (5, 5), 1)
        template_keypoints, template_descriptors = sift.detectAndCompute(template_gray_image, None)


        knn_matches = bf.knnMatch(template_descriptors, scene_descriptors, k = 2)
        good = []
        for m, n in knn_matches:
            if m.distance < 0.7 * n.distance:
                good.append([m])
        
        
        if len(good) > len(max_template_matches):
            max_template_matches = good
            max_template_rgb_image = template_rgb_image
            max_template_keypoints = template_keypoints


    img3 = cv2.drawMatchesKnn(max_template_rgb_image, max_template_keypoints, scene_rgb_image, scene_keypoints, max_template_matches, None)
    plt.imshow(img3)
    plt.show()
    
    cv2.imwrite('./images/q3_2_match' + str(testing_image_number) + '.jpg', img3)
    testing_image_number += 1